<a href="https://colab.research.google.com/github/coldsober-irene/dipulearning/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import os, cv2, shutil
from matplotlib import pyplot as plt
import random, pickle, numpy as np

In [6]:
dir = '/content/drive/MyDrive/Dataset'
categories = [category for category in os.listdir(dir)]
imsize = 220
print("CATEGORIES: ", categories)
images_needed = []
for category in categories:
  if '.' not in category:
    cat_path = os.path.join(dir, category)
    class_ = categories.index(category)
    for im in os.listdir(cat_path):
      image = os.path.join(cat_path, im)
      im_array = cv2.imread(image, cv2.IMREAD_GRAYSCALE)
      same_size = cv2.resize(im_array, (imsize, imsize))
      images_needed.append([same_size, class_])
      # plt.imshow(same_size, cmap = 'gray')
      # plt.show()

random.shuffle(images_needed)
train_data = np.array([row[0] for row in images_needed]).reshape(-1, imsize, imsize, 1)
labels = np.array([row[1] for row in images_needed])



CATEGORIES:  ['Car', 'Auto', 'Motorcycle', '.ipynb_checkpoints', 'train.pickle', 'labels.pickle']


In [7]:

with open(os.path.join(dir,"train.pickle"), 'wb') as f:
  pickle.dump(train_data, f)

with open(os.path.join(dir,"labels.pickle"), 'wb') as f:
  pickle.dump(labels, f)

In [8]:

train = pickle.load(open(os.path.join(dir,"train.pickle"), 'rb'))
lab = pickle.load(open(os.path.join(dir,"labels.pickle"), 'rb'))

train = np.array(train)
train = train / 255


In [9]:
train.shape, lab.shape

((1326, 220, 220, 1), (1326,))

In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Activation, Flatten
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.utils import to_categorical


#(batch_size, height, width, channels)

In [11]:
model = Sequential()
# LAYER 1
model.add(Conv2D(64, (3, 3), input_shape = train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
# LAYER 2
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
# LAYER 3
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
# LAYER 4
model.add(Flatten())
model.add(Dense(128))

# LAYER 5
model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(optimizer = 'adam',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])
lab = to_categorical(lab)
model.fit(train, lab,batch_size=32, validation_split=0.2, epochs = 10)

Epoch 1/10
34/34 [==============================] - 12s 133ms/step - loss: 1.5078 - accuracy: 0.5868 - val_loss: 0.6632 - val_accuracy: 0.7406
Epoch 2/10
34/34 [==============================] - 3s 85ms/step - loss: 0.4485 - accuracy: 0.8217 - val_loss: 0.5845 - val_accuracy: 0.7256
Epoch 3/10
34/34 [==============================] - 3s 89ms/step - loss: 0.2678 - accuracy: 0.9019 - val_loss: 0.4467 - val_accuracy: 0.8459
Epoch 4/10
34/34 [==============================] - 3s 88ms/step - loss: 0.1601 - accuracy: 0.9387 - val_loss: 0.4104 - val_accuracy: 0.8722
Epoch 5/10
34/34 [==============================] - 3s 86ms/step - loss: 0.0494 - accuracy: 0.9849 - val_loss: 0.4522 - val_accuracy: 0.8797
Epoch 6/10
34/34 [==============================] - 3s 88ms/step - loss: 0.0191 - accuracy: 0.9962 - val_loss: 0.5359 - val_accuracy: 0.8797
Epoch 7/10
34/34 [==============================] - 3s 85ms/step - loss: 0.0127 - accuracy: 0.9972 - val_loss: 0.4866 - val_accuracy: 0.8947
Epoch 8/10
